# Numerical experiments for *A sharp oracle inequality for Graph-Slope* by P. Bellec, J. Salmon and S. Vaiter

This Python notebook contains the code necessary to reproduce all the figures included in the paper *A sharp oracle inequality for Graph-Slope* by P. Bellec, J. Salmon and S. Vaiter.

## Setup

Imports and RNG setup

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pickle

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import osmnx as ox
from sklearn.metrics import mean_squared_error, precision_score
from scipy.linalg import pinv2

from graphslope import GraphSlope
from graphslope.utils import FDR, TDR

np.random.seed(44)
recompute = True

Figure style

In [ ]:
sns.set_context('paper', font_scale=1.4)
sns.set_style("ticks")
sns.set_palette("colorblind", 8)

Common funs

In [ ]:
def generate_random_sparse(D, n0):
    (n,p) = D.shape
    DTJ = np.array(D.T[np.random.permutation(p)[:p-n0],:].todense())
    gen = np.random.randn(n)
    U,S,Vt = np.linalg.svd(DTJ,full_matrices=False)
    idx_to_keep = S>1e-7
    x0 = gen - Vt[idx_to_keep,:].T.dot(Vt[idx_to_keep,:].dot(gen))
    return x0

In [ ]:
def run_xp(D, estimators, number_of_simulations=10, coeff=1.0):
    (n,p) = D.shape
    
    num_sparsities = np.zeros((p,1))
    MSEs = np.zeros((p, len(estimators)))
    FDRs = np.zeros((p, len(estimators)))
    TDRs = np.zeros((p, len(estimators)))

    for n0 in range(0,p):
        for j in range(number_of_simulations):
            x0 = generate_random_sparse(D, n0)
            x0 = coeff * sigma * x0
            y = x0 + sigma * np.random.randn(n)

            sparsity = int(np.sum(np.abs(D.T.dot(x0)) > 1e-4))
            num_sparsities[sparsity, 0] += 1

            for est_idx, est in enumerate(estimators):
                est.fit(y)
                xres = est.coef_
                MSEs[sparsity, est_idx] += mean_squared_error(xres, x0)
                FDRs[sparsity, est_idx] += FDR(D, xres, x0)
                TDRs[sparsity, est_idx] += TDR(D, xres, x0)

    MSEs /= num_sparsities
    FDRs /= num_sparsities
    TDRs /= num_sparsities
    
    return (MSEs, FDRs, TDRs)

In [ ]:
def rho(D):
    penrose_DT = pinv2(D.T.todense())
    return np.max(np.linalg.norm(penrose_DT,axis=1)) / np.sqrt(D.shape[0])

## Synthetic experiments for Caveman-like graph

Caveman graph generation

In [ ]:
nodes_by_component = 10
num_of_components = 4
probability_of_connect = 0.1

G = nx.relaxed_caveman_graph(num_of_components, nodes_by_component, probability_of_connect, seed=4)
D = nx.incidence_matrix(G, oriented=True)
(n, p) = D.shape

print((n,p))

In [ ]:
def nth_root_unit(n):
    t = np.arange(0,2.0*np.pi,2.0*np.pi/n,dtype=np.float32)
    return np.array([np.cos(t),np.sin(t)]).T

relative_distance = 2.0
nodes_relative_coordinates = nth_root_unit(nodes_by_component)
center_of_components = nth_root_unit(num_of_components)

pos = np.concatenate(list(map(lambda row: nodes_relative_coordinates + relative_distance*row, center_of_components)))
ec = ['k' if u//nodes_by_component == v//nodes_by_component else 'b' for u,v in G.edges()]

nx.draw_networkx(G,pos=pos,edge_color=ec,with_labels=False,node_size=100)
plt.axis('equal')
plt.axis('off')

plt.savefig("images/caveman_graph.pdf")

In [ ]:
sigma = 0.2
weights_slope = sigma * rho(D) * np.sqrt(2 * np.log(p/np.arange(1,p+1))*(n))
weights_lasso = sigma * rho(D) * np.sqrt(2 * np.log(p)*(n)) * np.ones(p)

from scipy.linalg import pinv2
penrose_D = pinv2(D.T.todense())
weights_slope_pen = np.percentile( -np.sort(-np.abs(penrose_D.T.dot(np.random.normal(size=(n,100000)))), axis=0 ), 99, axis=1)
weights_slope_pen *= rho(D) * sigma * np.sqrt(2 * np.log(p)*(n)) / weights_slope_pen[0]

In [ ]:
plt.plot(weights_lasso, label="Graph-Lasso")
plt.plot(weights_slope, label="Graph-Slope")
plt.plot(weights_slope_pen, label="Graph-Slope (MC)")
plt.legend()

plt.savefig("images/caveman_weight.pdf")

In [ ]:
glasso_reg = GraphSlope(incidence=D, lambdas=weights_lasso, max_iter=30000, tol=1e-5)
gslope_reg = GraphSlope(incidence=D, lambdas=weights_slope, max_iter=30000, tol=1e-5)
gslope_pen_reg = GraphSlope(incidence=D, lambdas=weights_slope_pen, max_iter=30000, tol=1e-5)
estimators = [glasso_reg, gslope_reg, gslope_pen_reg]

if recompute:
    MSEs, FDRs, TDRs = run_xp(D, estimators, number_of_simulations=1000, coeff=8)
    with open("output/caveman.p", "wb") as f:
        pickle.dump((MSEs, FDRs, TDRs), f)
else:
    with open("output/caveman.p", "rb") as f:
        MSEs, FDRs, TDRs = pickle.load(f)

In [ ]:
plt.figure()
plt.plot(range(p), MSEs)
plt.xlabel('$D^{\\top}$-sparsity')
plt.ylabel('MSE')
plt.legend(["Graph-Lasso", "Graph-Slope", "Graph-Slope (MC)"])

plt.savefig("images/caveman-s02-c8-mse.pdf")

In [ ]:
plt.figure()
plt.plot(range(1,p), FDRs[1:,:])
plt.xlabel('$D^{\\top}$-sparsity')
plt.ylabel('FDR')
plt.legend(["Graph-Lasso", "Graph-Slope", "Graph-Slope (MC)"])

plt.savefig("images/caveman-s02-c8-fdr.pdf")

In [ ]:
plt.figure()
plt.plot(range(1,p), TDRs[1:,:])
plt.ylim([0,1])
plt.xlabel('$D^{\\top}$-sparsity')
plt.ylabel('TDR')
plt.legend(["Graph-Lasso", "Graph-Slope", "Graph-Slope (MC)"])

plt.savefig("images/caveman-s02-c8-tdr.pdf")

## Synthetic experiments for TV-1D

In [ ]:
n = 100

G = nx.path_graph(n)
D = nx.incidence_matrix(G, oriented=True)
(n, p) = D.shape

print((n,p))

In [ ]:
sigma = 0.6
weights_slope = sigma * rho(D) * np.sqrt(2 * np.log(p/np.arange(1,p+1)) * (n))
weights_lasso = sigma * rho(D) * np.sqrt(2 * np.log(p) * (n)) * np.ones(p)

from scipy.linalg import pinv2
penrose_D = pinv2(D.T.todense())
weights_slope_pen = np.percentile( -np.sort(-np.abs(penrose_D.T.dot(np.random.normal(size=(n,100000)))), axis=0 ), 99, axis=1)
weights_slope_pen *= rho(D) * sigma * np.sqrt(2 * np.log(p) * (n)) / weights_slope_pen[0]

In [ ]:
plt.plot(weights_lasso, label="Graph-Lasso")
plt.plot(weights_slope, label="Graph-Slope")
plt.plot(weights_slope_pen, label="Graph-Slope (MC)")
plt.legend()

plt.savefig("images/tv1d-weight.pdf")

In [ ]:
glasso_reg = GraphSlope(incidence=D, lambdas=weights_lasso, max_iter=30000, tol=1e-5)
gslope_reg = GraphSlope(incidence=D, lambdas=weights_slope, max_iter=30000, tol=1e-5)
gslope_pen_reg = GraphSlope(incidence=D, lambdas=weights_slope_pen, max_iter=30000, tol=1e-5)
estimators = [glasso_reg, gslope_reg, gslope_pen_reg]

if recompute:
    MSEs, FDRs, TDRs = run_xp(D, estimators, number_of_simulations=100, coeff=8.0)
    with open("output/tv1d.p", "wb") as f:
        pickle.dump((MSEs, FDRs, TDRs), f)
else:
    with open("output/tv1d.p", "rb") as f:
        MSEs, FDRs, TDRs = pickle.load(f)

In [ ]:
plt.figure()
plt.plot(range(1,p), MSEs[1:,:])
plt.xlabel('$D^{\\top}$-sparsity')
plt.ylabel('MSE')
plt.legend(["Graph-Lasso", "Graph-Slope", "Graph-Slope (MC)"])

plt.savefig("images/tv1d-s1-c1-mse.pdf")

In [ ]:
plt.figure()
plt.plot(range(1,p), FDRs[1:,:])
plt.xlabel('$D^{\\top}$-sparsity')
plt.ylabel('FDR')
plt.legend(["Graph-Lasso", "Graph-Slope", "Graph-Slope (MC)"])

plt.savefig("images/tv1d-s1-c1-fdr.pdf")

In [ ]:
plt.figure()
plt.plot(range(1,p), TDRs[1:,:])
plt.ylim([0,1])
plt.xlabel('$D^{\\top}$-sparsity')
plt.ylabel('TDR')
plt.legend(["Graph-Lasso", "Graph-Slope", "Graph-Slope (MC)"])

plt.savefig("images/tv1d-s1-c1-tdr.pdf")

In [ ]:
np.random.seed(44)
xs = 8 * generate_random_sparse(D, 3)
y = xs + 0.6 * np.random.randn(D.shape[0])

glasso_reg = GraphSlope(incidence=D, lambdas=weights_lasso, max_iter=30000, tol=1e-5)
gslope_reg = GraphSlope(incidence=D, lambdas=weights_slope, max_iter=30000, tol=1e-5)
gslope_pen_reg = GraphSlope(incidence=D, lambdas=weights_slope_pen, max_iter=30000, tol=1e-5)

glasso_reg.fit(y)
gslope_reg.fit(y)
gslope_pen_reg.fit(y)

plt.plot(glasso_reg.coef_, label='Graph-Lasso')
plt.plot(gslope_reg.coef_, label='Graph-Slope')
plt.plot(gslope_pen_reg.coef_, label='Graph-Slope (MC)')

plt.plot(xs, 'k', label='True signal')
plt.plot(y, 'b.', label='Noisy signal')
plt.legend()

plt.savefig("images/tv1d-s1-c1-ex.pdf")

## Road network

Get data from OpenSteetMap (thanks to osmnx): we restrict our attention to road where cars can drive (a bit smaller than the full map).

In [ ]:
G = ox.graph_from_place('Paris, France', network_type='drive')
ox.plot_graph(G)

In [ ]:
# G is a MultiDiGraph, but we can use its incidence matrix as for any graph
D = nx.incidence_matrix(G, oriented=True)
(n,p) = D.shape
print("Dimension (n,p) = " + str((n,p)))

In [ ]:
def infect(G, i0, steps):
    x = np.zeros(n)
    x[i0] = 1.
    for s in range(steps):
        for i, node in enumerate(G.nodes()):
            if x[i] > 0:
                for neigh in G.neighbors(node):
                    if np.random.rand() > 0.25:
                        j = list(G.nodes()).index(neigh)
                        x[j] = 1.
    return x

np.random.seed(44)
k = 30
steps = 8
seeds = np.random.permutation(n)[:k]
x0 = infect(G, seeds, steps)

sigma = 0.8
y = x0 + sigma * np.random.randn(n)

In [ ]:
import matplotlib

def rescale_color(x, xmin, xmax, cmap='plasma'):
    #x_scaled = (xmax - x + x.mean())/(xmax - xmin)
    x_scaled = x
    bins, values = np.histogram(x,bins=np.linspace(xmin,xmax,num=254))# np.histogram(x_scaled,bins=7)
    cms = np.array(plt.get_cmap(cmap).colors)
    colors = np.digitize(x_scaled, values)
    colors = cms[colors,:]
    
    hexa = []
    for i in range(x_scaled.shape[0]):
        hexa.append(matplotlib.colors.rgb2hex(colors[i,:]))
    return hexa

In [ ]:
ox.plot_graph(G, node_size=20, node_color=rescale_color(x0, x0.min(), x0.max()), edge_alpha=0.4)

In [ ]:
glasso_reg = GraphSlope(incidence=D, lambdas=weights_lasso, max_iter=5000, tol=1e-1)
gslope_reg = GraphSlope(incidence=D, lambdas=weights_slope, max_iter=5000, tol=1e-1)
ran = np.logspace(-5,1.5,num=30)

best_fac_s_idx = -1
best_fac_s_mse = np.Inf
best_fac_l_idx = -1
best_fac_l_mse = np.Inf

for i, fac in enumerate(ran):
    glasso_reg.lambdas = sigma * fac * np.sqrt(2 * np.log(p)) * np.ones(p)
    glasso_reg.fit(y)
    xstar_l = glasso_reg.coef_
    if mean_squared_error(xstar_l,x0) < best_fac_l_mse:
        best_fac_l_idx = i
        best_fac_l_mse = mean_squared_error(xstar_l,x0)
    
    gslope_reg.lambdas = sigma * fac * np.sqrt(2 * np.log(p/np.arange(1,p+1)))
    gslope_reg.fit(y)
    xstar_s = gslope_reg.coef_
    if mean_squared_error(xstar_s,x0) < best_fac_s_mse:
        best_fac_s_idx = i
        best_fac_s_mse = mean_squared_error(xstar_s,x0)
        
print("Best lasso idx: {}".format(best_fac_l_idx))
print("Best slope idx: {}".format(best_fac_s_idx))

In [ ]:
weights_slope = ran[best_fac_s_idx] * sigma * np.sqrt(2 * np.log(p/np.arange(1,p+1))) # weights using logs
weights_lasso = ran[best_fac_l_idx] * sigma * np.sqrt(2 * np.log(p)) * np.ones(p)
glasso_reg = GraphSlope(incidence=D, lambdas=weights_lasso, max_iter=100000, tol=1e-5)
gslope_reg = GraphSlope(incidence=D, lambdas=weights_slope, max_iter=100000, tol=1e-5)

In [ ]:
glasso_reg.fit(y)
xstar_l = glasso_reg.coef_

gslope_reg.fit(y)
xstar_s = gslope_reg.coef_

xmin = np.min([x0.min(), y.min(), xstar_l.min(), xstar_s.min()])
xmax = np.max([x0.max(), y.max(), xstar_l.max(), xstar_s.max()])
xmin = - np.max([np.abs(xmin),np.abs(xmax)])
xmax = np.max([np.abs(xmin),np.abs(xmax)])

In [ ]:
print("\tGraph-Lasso\t\tGraph-Slope")
print("MSE\t{0}\t{1}".format(mean_squared_error(xstar_l,x0), mean_squared_error(xstar_s,x0)))
print("FDR\t{0}\t{1}".format(FDR(D,xstar_l,x0), FDR(D,xstar_s,x0)))
print("TDR\t{0}\t{1}".format(TDR(D,xstar_l,x0), TDR(D,xstar_s,x0)))

In [ ]:
xmin = np.min(list(map(lambda x: np.percentile(x,5),[x0,y,xstar_l,xstar_s])))
xmax = np.max(list(map(lambda x: np.percentile(x,95),[x0,y,xstar_l,xstar_s])))

In [ ]:
ox.plot_graph(G, node_size=20, node_color=rescale_color(x0, xmin, xmax), edge_alpha=0.4, save=True, close=True, show=True, file_format='pdf', filename='paris-x0')
ox.plot_graph(G, node_size=20, node_color=rescale_color(y, xmin, xmax), edge_alpha=0.4, save=True, close=True, show=True, file_format='pdf', filename='paris-y')
ox.plot_graph(G, node_size=20, node_color=rescale_color(xstar_l, xmin, xmax), edge_alpha=0.4, save=True, close=True, show=True, file_format='pdf', filename='paris-lasso')
ox.plot_graph(G, node_size=20, node_color=rescale_color(xstar_s, xmin, xmax), edge_alpha=0.4, save=True, close=True, show=True, file_format='pdf', filename='paris-slope')

In [ ]:
def cosp(D, x, tol=1e-6):
    return np.sum(np.abs(D.T.dot(x)) > tol)
cosp(D, x0), cosp(D, y), cosp(D, xstar_l), cosp(D, xstar_s)

In [ ]:
plt.imshow([xstar_s,x0], cmap='plasma')
plt.colorbar(orientation='horizontal')
plt.savefig("images/paris-colormap.pdf")